# Regression Model Comparison

In this exercise, you will compare the ability of various regression models to generalize across years.

## Setup

First, the following cells import required Python modules and define some functions that will be useful for the analysis:

In [1]:
# Imports
%env MPLCONFIGDIR=/tmp/.cache
import os
import numpy as np
import xarray as xr
from tqdm.notebook import tqdm
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut

env: MPLCONFIGDIR=/tmp/.cache


In [2]:
# Constants
DATA_DIRECTORY = '/home/jovyan/shared/NASA_Summer_School_2024/tree_mortality/materials/data'

# Data files contain flattened, tabular representation of data for model training
DATA_FILE = os.path.join(DATA_DIRECTORY, 'tree_mortality_gridded_training_nonzero.nc4')

TRAINING_VARIABLES = [
    'PR1-1', 'PRET1-1',
    'SPI1-1', 'SPI1-2', 'SPI1-3', 'SPI1-4','SPI1-5', 'SPI1-6',
    'SPEI1-1', 'SPEI1-2', 'SPEI1-3', 'SPEI1-4','SPEI1-5', 'SPEI1-6',
    'SPI2-1', 'SPI2-2', 'SPI2-3', 'SPI2-4','SPI2-5', 'SPI2-6',
    'SPEI2-1', 'SPEI2-2', 'SPEI2-3', 'SPEI2-4','SPEI2-5', 'SPEI2-6',
]

In [3]:
# Function Definitions

def filter_inf(X, y=None, fill=1e2):
    """
    Returns a version of the dataset in which infinite values have been
    replaced with a given finite fill value
    """
    if np.any(np.isnan(X)):
        raise ValueError('NaN entries present but not handled')
        
    Xnew = np.nan_to_num(X, neginf=-fill, posinf=fill)
    
    return Xnew if y is None else (Xnew, y)


def load_training_set(datafile, target='tpa'):
    """
    Loads a training dataset from the specified file, selecting a specific
    year if given.

    @param datafile: NetCDF4 data file path to load
    @param target: the target variable of the regression
        (default: tpa, trees per acre)

    @return (X, y, years), where X is the feature matrix, y are the
        mortality labels, and years are separate years in the dataset
    """

    # Open dataset and select year (if specified)
    ds = xr.open_dataset(datafile)

    # Drop all variables except for the desired features
    X = ds[TRAINING_VARIABLES].to_array().T

    # Get target mortality values
    y = ds[target].as_numpy()

    # Filter infinite values
    X, y = filter_inf(X, y)

    # Get group definitions (years)
    years = ds.year.values.astype(int)
    
    return X, y, years


def get_test_year(years, test_idx):
    # Check that there is only a single unique year represented in the test set, and return it
    test_year = np.unique(years[test_idx])
    assert len(test_year) == 1
    return test_year[0]


def print_results(results, metric=root_mean_squared_error):
    for year, true_pred in sorted(results.items()):
        score = metric(*true_pred)
        print(f'{year}: {score:.2f}')

    all_results = np.vstack([
        np.column_stack([true, pred]) for true, pred in results.values()
    ])
    overall_score = metric(all_results[:, 0], all_results[:, 1])
    print(f'Overall Score: {overall_score:.2f}')

In [4]:
# Load the training set by year
X, y, years = load_training_set(DATA_FILE)

# This is a variant of the random forest classifier
# You can experiment with alternative models here
model = ExtraTreesRegressor(
    n_estimators=100, max_depth=4,
    bootstrap=True, n_jobs=-1
)

# Define leave-one-year-out splits
cross_val = LeaveOneGroupOut()
splits = list(cross_val.split(X, y, years))

results_by_year = {}

for train_idx, test_idx in tqdm(splits, 'Training'):
    test_year = get_test_year(years, test_idx)
    model.fit(X[train_idx], y[train_idx])
    y_pred = model.predict(X[test_idx])
    y_true = y[test_idx]
    results_by_year[test_year] = (y_true, y_pred)

Training:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
print_results(results_by_year)

2012: 5.01
2013: 4.57
2014: 6.85
2015: 9.88
2016: 18.86
2017: 15.25
2018: 14.78
2019: 8.46
2021: 10.08
Overall Score: 13.50


## Questions

Using the code above as an example, implement and evaluate at least 3 different regression models from [scikit-learn](https://scikit-learn.org/stable/supervised_learning.html), and at least 1 additional "baseline" model (e.g., simple linear regression). There are functions above to produce root mean squared error (RMSE) scores for each held-out year.

1. Which models exhibit the most success in predicting mortality in the held-out year?
2. What factors might explain the difference in model performance across years?
3. There is a phenomenon in statistical machine learning called the ["bias-variance tradeoff"](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff), in which more complex models are better able to fit arbitrary relationships between the input variables and target values (i.e., they exhibit less "bias" towards a particular, for example linear, relationship), but this also leads to more "variance" in performance due to over-fitting to the particular random sample of data in the training set. Some models have parameters that control the complexity of the relationships that can be learned, such as the `max_depth' parameter of the random forest. Do you observe a bias-variance tradeoff as you explore different models and parameters for this problem? Provide some examples of instances where you observe the effect and how you mitigated it.